In [1]:
import numpy as np 

In [2]:
DNA_SIZE = 24
POP_SIZE = 200
CROSSOVER_RATE = 0.8
MUTATION_RATE = 0.005
N_GENERATIONS = 50
X_BOUND = [-3, 3]
Y_BOUND = [-3, 3]

In [6]:
def F(x,y):
    return 3 * (1 - x)**2 * np.exp(-(x**2) - (y + 1)**2)

def translateDNA(pop):
    x_pop = pop[:,1::2]
    y_pop = pop[:,::2]
    x = x_pop.dot(2**np.arange(DNA_SIZE)[::-1])/ float(2**DNA_SIZE - 1) * (X_BOUND[1] - X_BOUND[0]) + X_BOUND[0]
    y = y_pop.dot(2**np.arange(DNA_SIZE)[::-1])/ float(2**DNA_SIZE - 1) * (Y_BOUND[1] - Y_BOUND[0]) + Y_BOUND[0]
    return x,y

def get_best_individual(pop):
    fitnesses = get_fitness(pop)
    best_idx = np.argmax(fitnesses)
    x,y = translateDNA(pop)
    return x[best_idx],y[best_idx]

def get_fitness(pop):
    x,y = translateDNA(pop)
    pred = F(x,y)
    return (pred - np.min(pred)) + 1e-3
 
def select(pop, fitness):
    idx = np.random.choice(np.arange(POP_SIZE), size=POP_SIZE, replace=True, p=fitness/fitness.sum())
    return pop[idx]

def crossover_and_mutation(pop, CROSSOVER_RAGE = 0.8):
    new_pop = []
    for father in pop:
        child = father
        if np.random.rand() < CROSSOVER_RAGE:
            mother = pop[np.random.randint(POP_SIZE)]
            cross_points = np.random.randint(low=0, high=DNA_SIZE*2)
            child[cross_points:] = mother[cross_points:]
        mutation(child)
        new_pop.append(child)
    return new_pop

def mutation(child, MUTATION_RATE = 0.003):
    if np.random.rand() < MUTATION_RATE:
        mutate_point = np.random.randint(0,DNA_SIZE)
        child[mutate_point] = child[mutate_point] ^ 1


def print_info(pop):
    fitnees = get_fitness(pop)
    max_fitness_index = np.argmax(fitnees)
    x,y = translateDNA(pop)
    print("最优秀的基因:",pop[max_fitness_index])
    print("(x,y):",x[max_fitness_index], y[max_fitness_index])
    print("适应度:",fitnees[max_fitness_index])



In [12]:
def main():
    pop = np.random.randint(2, size=(POP_SIZE, DNA_SIZE*2))
    for i in range(N_GENERATIONS):
        pop = np.array(crossover_and_mutation(pop,CROSSOVER_RATE))
        fitness = get_fitness(pop)
        pop = select(pop,fitness=fitness)
        x,y = get_best_individual(pop)
        best_val = F(x,y)
        #print(f"Generation {i+1}: Best Value = {best_val},x = {x},y = {y}")

    print_info(pop)

main()

最优秀的基因: [0 0 1 1 0 1 1 0 0 0 1 0 0 1 1 1 1 1 1 1 0 1 0 1 0 1 1 0 1 1 0 0 1 1 1 1 0
 1 0 0 1 1 1 1 0 1 0 1]
(x,y): -0.6567137036748947 -0.9896119826800813
适应度: 0.0021275952093638964
